In [1]:
%pip install --upgrade --quiet  annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install langchain
!pip install langchain_community
!pip install sentence-transformers
!pip install python-dotenv
!pip install langchain_core

In [305]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Annoy

embeddings_func = HuggingFaceEmbeddings(model_name="hkunlp/instructor-large")

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [306]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [307]:
loader = CSVLoader(file_path='/content/ajio.csv')
documents = loader.load()

In [308]:
len(documents)

1813

In [269]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [270]:
len(texts)

1813

In [309]:
vector_store_from_docs = Annoy.from_documents(documents, embeddings_func)

In [310]:
from dotenv import load_dotenv

load_dotenv()

True

In [311]:
! echo HUGGINGFACEHUB_API_TOKEN="token" >> .env

In [312]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature":0.1, "max_length":200}
)

In [313]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [314]:
from langchain.prompts.prompt import PromptTemplate
template_string="""
<system>\
As your helpful assistant, I will retrieve content and provide the relevant output for your questions.
</system>

content: {context}

<user> {user}</user>

answer:
"""
prompt = PromptTemplate.from_template(template=template_string)


In [323]:
query = "women tshirt"
docs = vector_store_from_docs.similarity_search_with_score(query,k=3)

In [316]:
# motorbike_emb = embeddings_func.embed_query("men shirt")
# docs=vector_store_from_docs.similarity_search_by_vector(motorbike_emb, k=3)

In [317]:
docs

[Document(page_content='\ufeffBrand: levis\nDescription: Polo T-shirt with Typography\nCategory_by_gender: Men\nDiscount Price (in Rs.): 700\nOriginal Price (in Rs.): 1,399\nColor: red', metadata={'source': '/content/ajio.csv', 'row': 326}),
 Document(page_content='\ufeffBrand: levis\nDescription: Cotton Polo T-shirt\nCategory_by_gender: Men\nDiscount Price (in Rs.): 1,091\nOriginal Price (in Rs.): 1,399\nColor: beige', metadata={'source': '/content/ajio.csv', 'row': 388}),
 Document(page_content='\ufeffBrand: lee-cooper\nDescription: Brand Print V-neck T-shirt\nCategory_by_gender: Men\nDiscount Price (in Rs.): 419\nOriginal Price (in Rs.): 599\nColor: mustard', metadata={'source': '/content/ajio.csv', 'row': 294})]

In [324]:
chain=prompt|llm|output_parser
result=chain.invoke({"context":docs, "user":query})

In [325]:
# Split the result by the delimiter "Document("
data_split = result.split("Document(")[1:]

# Extract relevant information from each split
parsed_data = []
for idx, item in enumerate(data_split, 1):
    # Extract the page content
    content_start_index = item.find("page_content=") + len("page_content=")
    content_end_index = item.find(", metadata=")
    page_content = item[content_start_index:content_end_index].strip()

    # Replace special characters with newline characters
    page_content = page_content.replace("\\n", "\n")

    # Append to parsed_data list
    parsed_data.append((idx, page_content))

# Print the extracted data
for idx, data in parsed_data:
    print(f"{idx}. {data}\n")


1. '\ufeffBrand: marie-claire
Description: Graphic T-shirt with Round Neckline
Category_by_gender: Women
Discount Price (in Rs.): 1,295
Original Price (in Rs.): 1,299
Color: black'

2. '\ufeffBrand: madame
Description: Full Sleeves Top with Printed Neck Tie-Up
Category_by_gender: Women
Discount Price (in Rs.): 800
Original Price (in Rs.): 1,599
Color: black'

3. '\ufeffBrand: only
Description: Typographic Print Crew-Neck T-shirt
Category_by_gender: Women
Discount Price (in Rs.): 1,299
Original Price (in Rs.): 1,299
Color: black'

